# <center> Reconnaissance visuelle 

### 1. Préparation du dataset pour vision

On choisit d'utiliser le dataset avec les transformations pertinentes appliquées après exploration des données. Notamment 4 styles avec un nombre insuffisant d'image ont été supprimés pour avoir des classes équilibrées lors de l'apprentissage dans la partie Deep Learning.

In [3]:
import sys
from pathlib import Path

# Pour que notre archi fonctionne avec google colab 
    
!git clone https://github.com/julietteabalain-cloud/Reconnaissance-de-mouvement-artistique.git
!cd /content/Reconnaissance-de-mouvement-artistique && git pull
%cd /content/Reconnaissance-de-mouvement-artistique 
import sys
sys.path.append(".")  # pour que src/ soit importable

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

Cloning into 'Reconnaissance-de-mouvement-artistique'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 196 (delta 122), reused 136 (delta 62), pack-reused 0 (from 0)
Receiving objects: 100% (196/196), 16.56 MiB | 29.70 MiB/s, done.
Resolving deltas: 100% (122/122), done.
Already up to date.
/content/Reconnaissance-de-mouvement-artistique


In [4]:
import torch
from src.dataset import load_df_train_test_val, load_df, load_image
from src.preprocessing import clean_dataset

#pour avoir acces au GPU si dispo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [6]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path

PROJECT_ROOT = Path("/content/deepl-projet")
DATA_ROOT = Path("/content/drive/MyDrive/DeepLearning/WikiArt_Subset")


df_test, df_train, df_val = load_df_train_test_val(DATA_ROOT)
df_full = load_df(DATA_ROOT)

df, df_train, df_val, df_test = clean_dataset(df_full, df_train, df_val, df_test)

print(f"Vérification des splits :")
print(f"- Train: {df_train['split'].unique()} ({len(df_train)} images)")
print(f"- Val  : {df_val['split'].unique()} ({len(df_val)} images)")
print(f"- Test : {df_test['split'].unique()} ({len(df_test)} images)")


Mounted at /content/drive
Vérification des splits :
- Train: ['train'] (6449 images)
- Val  : ['val'] (1373 images)
- Test : ['test'] (1378 images)


/content/Reconnaissance-de-mouvement-artistique/src/preprocessing.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["style_encoded"] = le.fit_transform(df_train["style"])
/content/Reconnaissance-de-mouvement-artistique/src/preprocessing.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val["style_encoded"]   = le.transform(df_val["style"])
/content/Reconnaissance-de-mouvement-artistique/src/preprocessing.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [7]:
import src.preprocessing as preprocessing_module
import src.vision_features_extract as vision_features_extract_module

from importlib import reload

vision_features_extract_module = reload(vision_features_extract_module)

### 2. Extraction des features



#### 2.1 Contour et gradients 



Dans cette section, nous étudions les contours présents dans les œuvres du dataset WikiArt.
L’objectif est de déterminer si certains styles artistiques se distinguent par des contours
plus nets ou plus flous, à l’aide d’un détecteur de Sobel et d’analyses statistiques des gradients.

L'intuition serait que les styles nets (Cubisme, Pop Art, Precisionism) auraient des gradients forts, orientations bien définies, forte magnitude tandis que les styles flous (Impressionism, Romanticism, Baroque) auraient des gradients plus faibles, orientations dispersées, faible magnitude.

##### 2.1.1 Sous Dataset

Nous sélectionnons quelques styles contrastés afin de faciliter l’analyse qualitative
et quantitative des contours.


In [8]:
selected_styles = ["Impressionism", "Cubism", "Baroque", "Romanticism", "Pop_Art"]

df_subset = df[df["style_name"].isin(selected_styles)]
print(df_subset["style_name"].value_counts())


style_name
Pop_Art          400
Cubism           400
Impressionism    400
Baroque          400
Romanticism      400
Name: count, dtype: int64


##### 2.1.2 Sobel

In [10]:
vision_features_extract_module.plot_sobel_examples(
    df_subset,
    load_image,
    DATA_ROOT,
    styles=selected_styles,
    n_images_per_style=3
)


TypeError: plot_sobel_examples() got multiple values for argument 'styles'

On observe que certains styles présentent des contours plus nets et structurés
(PopArt, Cubism), tandis que d’autres styles comme l’Impressionnisme montrent
des transitions plus douces et moins de contours marqués.


##### 2.1.3 Extraction des histogrammes de gradient

Nous calculons maintenant, pour chaque image, des histogrammes de magnitude
et d’orientation des gradients afin de quantifier la structure des contours.


In [ ]:
style_gradients = vision_features_extract_module.aggregate_histograms_by_style(
    df,
    load_image,
    DATA_ROOT,
    max_images_per_style=150
)


##### 2.1.4 Histogrammes moyens par styles

On affiche les histogrammes moyens par style de magnitude puis d'orientation :

In [ ]:
vision_features_extract_module.plot_mean_gradient_histograms(
    style_gradients,
    selected_styles,
)


On observe seulement pour Pop Art une démarcation dans l'histogramme des orientations du gradient. Le reste des styles ne sont pas particulièrement reconnaissables via cette information. Ce n'est pas suffisant.

On va essayer de claculer un score par style pour determiner si c'est uns tyle aux contour fort ou fiable

In [ ]:
vision_features_extract_module.compute_scores(selected_styles, style_gradients)

Impressionnisme = score élevé : beaucoup de coups de pinceau, gradients forts, mais diffus spatialement

Pop Art = score faible : aplats de couleur, grandes zones uniformes, peu de gradient

L’analyse des gradients de Sobel montre que les styles artistiques se
distinguent nettement par l’intensité moyenne de leurs transitions
locales.
Certains styles, comme l’impressionnisme ou l’expressionnisme,
présentent une forte proportion de gradients élevés, traduisant une
richesse de variations locales et de coups de pinceau visibles.
À l’inverse, des styles comme le Pop Art ou le Minimalisme reposent
davantage sur de larges aplats de couleur, générant moins de gradients
de forte amplitude.

#### 2.2 Descripteurs globaux

Nous utiliserons des descripteurs globaux pour
compl´eter nos features. Nous utiliserons le descripteur
HOG qui capturera la structure directionnelle locale (tech-
nique picturale) ainsi qu’un descripteur couleur dans les
espaces.


Pour chaque image on extrait HOG : Histogram of Oriented Gradient.
Puis on extrait des descripteurs de couleur dans les espaces Lab et HSV. 
Enfin on normalise et combine toutes ses features en un vecteur par image. 

In [ ]:
x, y = vision_features_extract_module.extract_features_dataset(df, load_image, DATA_ROOT)

On observe les résultats : 
D'abord en observant quels styles sont visuellement proches avec HOG + couleurs, et ensuite en observant le rendu PCA.

In [ ]:
dist_df = vision_features_extract_module.visualize_style_distances(x,y)

In [ ]:
vision_features_extract_module.heatmap_style_distances(dist_df)
# intéressant de comparer cette matrice a celle des distances avec les genres 

In [ ]:
vision_features_extract_module.pca_style_features(x,y)

### 3. Extraction des features en CSV

#### 3.1 StandardScaler + PCA

Etant donné la grande taille des vecteurs features pour chaque image (+14000), on décide d'utiliser standardscaler puis d'appliquer PCA pour réduire la dimension du vecteur feature tout en conservant les informations pertinentes.

In [ ]:
result = vision_features_extract_module.apply_pca_for_model(df_train, df_val, df_test, n_components=300)

X_train_pca = result["X_train"]
X_val_pca   = result["X_val"]
X_test_pca  = result["X_test"]

In [ ]:
import pandas as pd


df_train_pca = vision_features_extract_module.build_pca_dataframe(X_train_pca, df_train, "train")
df_val_pca   = vision_features_extract_module.build_pca_dataframe(X_val_pca, df_val, "val")
df_test_pca  = vision_features_extract_module.build_pca_dataframe(X_test_pca, df_test, "test")

df_all_pca = pd.concat([df_train_pca, df_val_pca, df_test_pca])



#### 3.2 Extraction en CSV

In [ ]:
df_all_pca.to_csv("/content/drive/MyDrive/dataset_vision_features.csv", index=False)


# todolist

## 2. Extraction de features
### 2.1. Contours et gradients
- [ ] Appliquer **détecteur de Sobel** sur chaque image DONE 
- [ ] Calculer histogrammes pour chaque image :DONE
  - Magnitude des gradients
  - Orientation des gradients
- [ ] Vérifier que les histogrammes différencient bien styles nets vs flous. BOF

### 2.2. Descripteurs globaux
- [ ] Extraire **HOG** (Histogram of Oriented Gradients) pour chaque image. DONE
- [ ] Extraire descripteurs de couleur dans les espaces **Lab** et **HSV**. DONE
- [ ] Normaliser et combiner toutes les features en un seul vecteur par image. DONE
- [ ] Créer une fonction `extract_features(image)` réutilisable.  DONE
- Avoir un bilan "palette de couleur" par styles  #TODO
 
## 3. Classification SVM
- [ ] Séparer features en train / val / test. DONE
- [ ] Normaliser / standardiser les features. + combiner tout les descripteurs precedents
- [ ] Entraîner un **SVM avec noyau RBF** sur le train.
- [ ] Évaluer le modèle sur val et test :
  - Précision globale
  - F1-score par style
  - Matrices de confusion détaillées
- [ ] Identifier les confusions fréquentes et vérifier si elles correspondent à des confusions humaines.
- [ ] Identifier les features les plus discriminantes pour chaque style.

## 4. Analyse et visualisation
- [ ] Visualiser quelques images par style pour valider les features.
- [ ] Histogrammes :
  - Nombre d’images par style
  - Distribution des artistes par style (exclure “unknown”)
  - Distribution des genres par style (exclure “unknown”)
- [ ] Analyse qualitative des styles avec images non “peinture” (statues, architecture…).
- [ ] Étudier la proportion d’artistes communs entre splits et risques de fuite.
- [ ] Visualiser relations artistes ↔ styles et genres ↔ styles.
- [ ] Préparer graphiques clairs pour le rapport (histogrammes, exemples d’images).

## 5. Validation causale par ablation (facultatif / si FFT fait par l’autre personne)
- [ ] Créer versions du dataset :
  - Original
  - Passe-haut
  - Passe-bas
- [ ] Entraîner un ResNet18 pour chaque version.
- [ ] Comparer performances pour confirmer l’importance des signatures fréquentielles.

## Notes
- Toutes les fonctions d’extraction doivent être centralisées dans `src/dataset.py`.
- Préparer les notebooks pour visualisation et tests rapides sans écraser les fichiers originaux.
